In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# relative imports
from uncertainty_aware_diagnosis import (
    split_and_save_csv,
)

# absolute imports
import polars as pl

# paths
train_csv = "./data/lbz-train.csv"
val_csv = "./data/lbz-val.csv"
test_csv = "./data/lbz-test.csv"
input_csv = "B:/Uitwissel DV en IP/Mike/Bestanden voor Luuk/Opnamen Luuk.csv"

In [ ]:
opnamen = pl.read_csv(input_csv)
opnamen.head()

In [ ]:
# split data into train, val, test
target = "icd10_main_code"

split_and_save_csv(
    input_csv=input_csv,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=test_csv,
    train_frac=0.7,
    val_frac=0.15,
    test_frac=0.15,
    stratify_col=target,  # preserve class balance
    random_state=42,
)